In [27]:
import pandas as pd
import vobject as vo
import phonenumbers
from phonenumbers import geocoder, carrier
from phonenumbers import NumberParseException
import os
from tqdm import tqdm

In [28]:
def define_loja(loja):
    
    indice_final = loja.find("_")
    indice_inicio =indice_final + 1

    for i in range(indice_inicio, len(loja)):
        if not loja[i].isdigit():
            indice_final = i
        else:
            break
    texto_extraido = loja[indice_inicio:indice_final + 1]
    return texto_extraido


def trata_nomes (nome):
    if len(nome) <= 1 or nome == None or nome == '' or nome == ' ':
        nome = "Nao Identificado"
        return nome
    else:
        nome = nome[1]
        nome = nome.strip()
        nome = nome.replace(';', ' ').replace(',', ' ').replace('-', ' ')
        #nome = nome.replace('â', 'a').replace('ã', 'a').replace('á', 'a').replace('à', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('ú', 'u').replace('ç', 'c').replace('Â', 'A').replace('Ã', 'A').replace('Á', 'A').replace('À', 'A').replace('É', 'E').replace('Ê', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ô', 'O').replace('Ú', 'U').replace('Ç', 'C')
        return nome

def trata_nomes_sistema (nome):
    if len(nome) <= 1 or nome == None or nome == '' or nome == ' ':
        nome = "Nao Identificado"
        return nome
    else:
        nome = nome.strip()
        nome = nome.replace(';', ' ').replace(',', ' ').replace('-', ' ').replace('+', ' ').replace('(', ' ').replace(')', ' ').replace('.', ' ').replace('/', ' ').replace('_', ' ').replace('*', ' ').replace(';', ' ').replace(',', ' ').replace('#', ' ').replace("$", ' ')
        nome = nome.replace('â', 'a').replace('ã', 'a').replace('á', 'a').replace('à', 'a').replace('é', 'e').replace('ê', 'e').replace('í', 'i').replace('ó', 'o').replace('ô', 'o').replace('ú', 'u').replace('ç', 'c').replace('Â', 'A').replace('Ã', 'A').replace('Á', 'A').replace('À', 'A').replace('É', 'E').replace('Ê', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ô', 'O').replace('Ú', 'U').replace('Ç', 'C')
        return nome

def trata_cel(cel):

    if (len(cel[1]) < 8 or cel is None or cel == '' or cel == ' ' or cel[1].isalpha() == True):
        return None
    else:
        cel = cel[1]
        cel = cel.strip()
        cel = cel.replace(' ','').replace('(','').replace(')','').replace('+','').replace('.','').replace('/','').replace('_','').replace('*', '').replace(';', '').replace(',', '').replace('#', '').replace("$", '').replace('-', '')
        if (cel.isalpha() == True or cel.isalnum() == False):
            return None
        if len(cel) == 9:
            cel = '021' + cel
        elif len(cel) == 8:
            cel = '0219' + cel
        return cel

def trata_cel_sistema(cel):

    if (len(cel) < 8 or cel is None or cel == '' or cel == ' ' or cel.isalpha() == True or cel[0:4] == '(00)'):
        return None
    else:
        #cel = cel[1]
        cel = cel.strip()
        cel = cel.replace(' ','').replace('(','').replace(')','').replace('+','').replace('.','').replace('/','').replace('_','').replace('*', '').replace(';', '').replace(',', '').replace('#', '').replace("$", '').replace('-', '')
        if (cel.isalpha() == True or cel.isalnum() == False):
            return None
        if len(cel) == 9:
            cel = '021' + cel
        elif len(cel) == 8:
            cel = '0219' + cel
        return cel

def parse_phone_number(tel, region='BR'):
    value = None
    parsed_tel = None 
    e164_format = None
    national = None
    international = None

    parsed_tel = phonenumbers.parse(tel, region)

    # Verifica se o número de telefone é válido
    if phonenumbers.is_valid_number(parsed_tel):
    
        # Transforma o número de telefone para o formato Nacional
        if phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.NATIONAL) is None:
            national = 0
        else:
            national = phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.NATIONAL)

        # Transforma o número de telefone para o formato E164
        if phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.E164) is None:
            e164_format = 0
        else:
            e164_format = phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.E164)

        # Transforma o número de telefone para o formato Internacional
        if phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.INTERNATIONAL) is None:
            international = 0
        else:
            international = phonenumbers.format_number(parsed_tel, phonenumbers.PhoneNumberFormat.INTERNATIONAL)

        
        if e164_format != 0:
            value = e164_format
        elif national != 0:
            value = national
        elif international != 0:
            value = international
        else:
            value = None

    else :
        value = None
        
    return value

#### Trata os arquivos dos celulares e junta os que sao da mesma loja.

In [33]:
#Caminho onde estao os dados
path = '../data/'
path_cel = '../output_celulares/'
lojas_inseridas = []

for nome_arquivo in tqdm(os.listdir(path), desc='Gerando arquivos'):

    #Caminho do arquivo
    caminho_arquivo = os.path.join(path, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)

    #Acrescenta o nome da loja na lista das lojas 
    print(f"Gerando arquivo {nome_arquivo_sem_extensao}.csv")

    #Exclui os valores do dataframe e recria o dataframe
    df = None
    #email = None
    df = pd.DataFrame(columns = ['Nome', 'email', 'Telefone', 'Loja'])

    if os.path.isfile(caminho_arquivo):
        with open(caminho_arquivo, 'r') as f:
            text = f.read()

            #Separando os contatos
            contacts = text.split('BEGIN:VCARD')[1:]

            #Percorrendo os contatos e extraindo os valores para uma lista
            contacts = [contact.split('\n') for contact in contacts]

            #Percorrendo os contatos e extraindo os valores para um dicionário
            for n,a in enumerate(contacts):
                for i in a:
                    # Inicia tratamento dos dados
                    if 'FN' in i:
                        parts_fn = i.split(':')
                        name = trata_nomes(parts_fn)
                    elif 'TEL;CEL' in i:
                        parts_cel = i.split(':')
                        tel = trata_cel(parts_cel)
                        if tel is not None:
                            parsed_tel = parse_phone_number(tel)
                        else:
                            parsed_tel = None
                if parsed_tel is not None:
                    email = ''
                    df.loc[n] = [name, email, parsed_tel, loja]

            #Salva o arquivo csv
            df.to_csv(f'../output_celulares/{nome_arquivo_sem_extensao}.csv', index=False)
            df = df.drop_duplicates(subset='Telefone', keep='first')
            print(f"Arquivo {nome_arquivo_sem_extensao}.csv gerado com sucesso!")
            print("=================")

#Unindo os arquivos
print("==============UNINDO OS ARQUIVOS================")
for nome_arquivo in tqdm(os.listdir(path_cel), desc='Unindo os arquivos'):

    #Caminho do arquivo
    caminho_arquivo = os.path.join(path_cel, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)

    if loja not in lojas_inseridas:

        #Insere a loja na lista
        lojas_inseridas.append(loja)

        #Atribuindo dataframe ao arquivo
        df = pd.read_csv(caminho_arquivo)
        df.drop_duplicates(subset='Telefone', keep='first')
        df.to_csv(f'../output_celulares_juntos/Contatos_{loja}.csv', index=False)
        print(f"Arquivo Contatos_{loja}.csv gerado com sucesso!")
        df = None
        print("=================")
    else:
        #Ler o arquivo atual
        df_new = pd.read_csv(caminho_arquivo)

        #Ler o dataframe existente e concatena os dois
        df_existing = pd.read_csv(f'../output_celulares_juntos/Contatos_{loja}.csv')  
        df = pd.concat([df_existing, df_new], ignore_index=True)

        #Remove duplicados
        df = df.drop_duplicates(subset='Telefone', keep='first')

        #Salva o arquivo resultante
        df.to_csv(f'../output_celulares_juntos/Contatos_{loja}.csv', index=False)
        print(f"Arquivo {nome_arquivo_sem_extensao}.csv adicionado com sucesso!")
        print("=================")

        #Zera as variáveis
        df = None
        df_new = None
        df_existing = None

Gerando arquivos:   0%|          | 0/40 [00:00<?, ?it/s]

Gerando arquivo Contatos_BarraShopping.csv


Gerando arquivos:   2%|▎         | 1/40 [00:06<04:14,  6.53s/it]

Arquivo Contatos_BarraShopping.csv gerado com sucesso!
Gerando arquivo Contatos_BarraShopping2.csv


Gerando arquivos:   5%|▌         | 2/40 [00:32<11:12, 17.70s/it]

...

Unindo os arquivos:  88%|████████▊ | 35/40 [00:04<00:00, 10.85it/s]

Arquivo Contatos_NorteShopping2.csv adicionado com sucesso!
Arquivo Contatos_NorteShopping3.csv adicionado com sucesso!
Arquivo Contatos_NovaIguacu.csv gerado com sucesso!
Arquivo Contatos_NovaIguacu2.csv adicionado com sucesso!
Arquivo Contatos_Tijuca.csv gerado com sucesso!


Unindo os arquivos:  95%|█████████▌| 38/40 [00:04<00:00, 12.16it/s]

Arquivo Contatos_Tijuca2.csv adicionado com sucesso!
Arquivo Contatos_Tijuca3.csv adicionado com sucesso!
Arquivo Contatos_Tijuca4.csv adicionado com sucesso!


Unindo os arquivos: 100%|██████████| 40/40 [00:04<00:00,  8.70it/s]


#### Trata os arquivos do sistema Estetica Ágil de cada loja

In [34]:
#Caminho onde estao os dados
path = '../dados_sistema/'

for nome_arquivo in tqdm(os.listdir(path), desc='Gerando arquivos do sistema'):
    #Caminho do arquivo
    caminho_arquivo = os.path.join(path, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)
    print(f'Tratando loja {loja}')

    df = None
    df = pd.DataFrame(columns = ['Nome', 'email', 'Telefone', 'Loja'])

    if os.path.isfile(caminho_arquivo):
        df_sistema = pd.read_excel(caminho_arquivo)
        #check null
#        print(df_sistema.isnull().sum())

        for i,a in enumerate(df_sistema['Telefone']):
            nome_sistema = trata_nomes_sistema(df_sistema['Nome'][i])
            email_sistema = df_sistema['E-mail'][i]
            tel = df_sistema['Telefone'][i]
            tel = trata_cel_sistema(str(tel))
            if tel is not None:
                parsed_tel = parse_phone_number(tel)
            else:
                parsed_tel = None
            

            if parsed_tel is not None:
                df.loc[i] = [nome_sistema, email_sistema, parsed_tel, loja]
                    
        #drop dupliactes by column Telefone
        df = df.drop_duplicates(subset='Telefone', keep='first')
        df.to_csv(f'../output_sistema/Contatos_{loja}.csv', index=False)
        print(f"Arquivo {loja}.csv gerado com sucesso!")

        #print(f"Aderencia da base de dados com a base de dados do sistema na loja {loja} foi de {df.shape[0]} de {df_sistema.shape[0]}. Isso significa que o % de aderencia foi de {round((df.shape[0]/df_sistema.shape[0])*100, 2)}%")
        #print(f"O número de registros que nao tiveram aderencia foi de {df_sistema.shape[0] - df.shape[0]}")

Tratando loja BarraShopping
Arquivo BarraShopping.csv gerado com sucesso!
Tratando loja BossaNova
Arquivo BossaNova.csv gerado com sucesso!
Tratando loja Dermatologic
Arquivo Dermatologic.csv gerado com sucesso!
Tratando loja DtwBarra
Arquivo DtwBarra.csv gerado com sucesso!
Tratando loja Esthetic
Arquivo Esthetic.csv gerado com sucesso!
Tratando loja Ipanema
Arquivo Ipanema.csv gerado com sucesso!
Tratando loja Laser
Arquivo Laser.csv gerado com sucesso!
Tratando loja Niteroi
Arquivo Niteroi.csv gerado com sucesso!
Tratando loja NorteShopping
Arquivo NorteShopping.csv gerado com sucesso!
Tratando loja Tijuca
Arquivo Tijuca.csv gerado com sucesso!


#### Cria lista com os celulares que cada loja possui

In [35]:
caminho_celulares = '../output_celulares_juntos/'
caminho_sistema = '../output_sistema/'
caminho_final = '../output_final/'

#Listas para armazenar as lojas
lojas_celulares = []
lojas_sistema = []

for nome_arquivo in os.listdir(caminho_celulares):
    
    #Caminho arquivo
    caminho_arquivo = os.path.join(caminho_celulares, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)
    lojas_celulares.append(loja)

for nome_arquivo in os.listdir(caminho_sistema):
    
    #Caminho arquivo
    caminho_arquivo = os.path.join(caminho_celulares, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)
    lojas_sistema.append(loja)

#### Consolida os dados do sistema e do celular em um unico arquivo por loja

In [36]:
#Juntando os arquivos das pastas de acordo com o nome da loja
caminho_celulares = '../output_celulares_juntos/'
caminho_sistema = '../output_sistema/'
caminho_final = '../output_final/'

for nome_arquivo in os.listdir(caminho_celulares):
    
    #Caminho do arquivo
    caminho_arquivo = os.path.join(caminho_celulares, nome_arquivo)
    
    #Nome do arquivo sem extensao
    nome_arquivo_sem_extensao = os.path.splitext(nome_arquivo)[0]

    #Nome da Loja
    loja = define_loja(nome_arquivo_sem_extensao)

    if loja in lojas_sistema:

        for nome_arquivo_sistema in os.listdir(caminho_sistema):
            #Caminho do arquivo
            caminho_arquivo_sistema = os.path.join(caminho_sistema, nome_arquivo_sistema)
            
            #Nome do arquivo sem extensao
            nome_arquivo_sem_extensao_sistema = os.path.splitext(nome_arquivo_sistema)[0]

            #Nome da Loja
            loja_sistema = define_loja(nome_arquivo_sem_extensao_sistema)

            if loja == loja_sistema:
                print(f'Juntando arquivos da loja {loja}')

                #Abrir o arquivo do nome loja contendo os dados dos celulares:
                df = pd.read_csv(caminho_arquivo)

                #Abrir o arquivo do nome loja contendo os dados do sistema:
                df_sistema = pd.read_csv(caminho_arquivo_sistema)

                #Juntando os arquivos
                df_final = pd.concat([df, df_sistema], ignore_index=True)
                df_final = df_final.drop_duplicates(subset='Telefone', keep='first')
                df_final.to_csv(f'{caminho_final}Contatos_{loja}.csv', index=False)
                print(f"Arquivo {loja}.csv gerado com sucesso!")
                print("===========================================")
    
    else:
        df = pd.read_csv(caminho_arquivo)
        df.to_csv(f'{caminho_final}Contatos_{loja}.csv', index=False)
        print(f"Arquivo {loja}.csv gerado com sucesso!")
        print("===========================================")

        

Juntando arquivos da loja BarraShopping
Arquivo BarraShopping.csv gerado com sucesso!
Juntando arquivos da loja BossaNova
Arquivo BossaNova.csv gerado com sucesso!
Juntando arquivos da loja Dermatologic
Arquivo Dermatologic.csv gerado com sucesso!
Juntando arquivos da loja DtwBarra
Arquivo DtwBarra.csv gerado com sucesso!
Arquivo DtwExperience.csv gerado com sucesso!
Arquivo DtwPremium.csv gerado com sucesso!
Juntando arquivos da loja Esthetic
Arquivo Esthetic.csv gerado com sucesso!
Juntando arquivos da loja Ipanema
Arquivo Ipanema.csv gerado com sucesso!
Juntando arquivos da loja Laser
Arquivo Laser.csv gerado com sucesso!
Juntando arquivos da loja Niteroi
Arquivo Niteroi.csv gerado com sucesso!
Juntando arquivos da loja NorteShopping
Arquivo NorteShopping.csv gerado com sucesso!
Arquivo NovaIguacu.csv gerado com sucesso!
Juntando arquivos da loja Tijuca
Arquivo Tijuca.csv gerado com sucesso!
